In [2]:
import pandas as pd

In [4]:
pothole_gov_df = pd.read_csv('../../data/raw/nyc_gov_pothole.csv')

In [5]:
pothole_311_df = pd.read_csv('../../data/raw/nyc_311_pothole.csv')

In [6]:
import ast

In [7]:
def calculate_average_coordinate(geometry_str):
    geometry = ast.literal_eval(geometry_str)
    total_lat, total_lng, count = 0,0,0

    for line in geometry['coordinates']:
        for coord_pair in line:
            total_lng += coord_pair[0]
            total_lat += coord_pair[1]
            count += 1
    centroid_lat = total_lat / count
    centroid_lng = total_lng / count
    
    return centroid_lng, centroid_lat

In [8]:
pothole_gov_df[['longitude','latitude']] = pothole_gov_df.apply(
    lambda row: calculate_average_coordinate(row['the_geom']),
    axis=1,
    result_type='expand'
)

In [9]:
import geopandas as gpd

In [10]:
pothole_gov_gdf = gpd.GeoDataFrame(
    pothole_gov_df,
    geometry=gpd.points_from_xy(pothole_gov_df.longitude, pothole_gov_df.latitude)
)

pothole_311_gdf = gpd.GeoDataFrame(
    pothole_311_df,
    geometry=gpd.points_from_xy(pothole_311_df.longitude, pothole_311_df.latitude)
)

In [11]:
pothole_311_gdf.crs = {"init": "epsg:4326"}
pothole_gov_gdf.crs = {"init": "epsg:4326"}

/Users/DonGuest/.local/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [12]:
projected = "EPSG:32618"
pothole_311_gdf = pothole_311_gdf.to_crs(projected)
pothole_gov_gdf = pothole_gov_gdf.to_crs(projected)

In [13]:
tolerance = 0.1
spatially_joined_gdf = gpd.sjoin_nearest(
    pothole_311_gdf, pothole_gov_gdf,
    max_distance=tolerance, distance_col='distance'
)

In [29]:
spatially_joined_gdf['closed_date'] = pd.to_datetime(spatially_joined_gdf['closed_date']).dt.normalize()
spatially_joined_gdf['RptClosed'] = pd.to_datetime(spatially_joined_gdf['RptClosed']).dt.normalize()

date_matched_df = spatially_joined_gdf[spatially_joined_gdf['closed_date'] == spatially_joined_gdf['RptClosed']]


In [31]:
date_matched_df.to_csv('../../data/processed/aligned_pothole.csv')